In [1]:
#dependencies
import re
import pandas as pd
import numpy as np
import scipy.stats as stats

import json
# import xlsxwriter

### Non Parametric Tests - Ordinal/ Nominal  DVs

In [ ]:
file_name = 'RawData.xlsx'
sheet_name = 'Survey Raw Data'
data_workbook = "SampleData.xlsx"
significant_val = 0.05
len_df = 499 # this should be infered from the data rather being assigned

# encoding headers ( not necessary unless there is a heirarchy of headers)
df = pd.read_excel(file_name, sheet_name = sheet_name)
df_heading = code_encode_headers(df)

# variable dictionary that stores call coded headers ( not necessary. only used as my data had heirarchy of headers)
with open('CodeJsonVar', 'r') as file:
    dic = json.load(file)

# storing resukts in a data frame
df_result = ONStatisticTests(data_workbook, dic, df_heading)

In [21]:
df_result.to_excel('SampleResults.xlsx', index=False)

### Basic Functions

In [9]:
def code_encode_headers(df):


    # gather all header and sub-header names from the three rows
    header_first =  [pd.NA if "Unnamed" in x else x for x in list(df.columns)]
    header_second = list(df.iloc[0])
    header_third = list(df.iloc[1])

    # create df from for the headings
    heading = {'header_first': header_first, 'header_second': header_second, 'header_third' : header_third}
    df_heading = pd.DataFrame(heading)
    df_heading[['header_first', 'header_second']] = df_heading[['header_first', 'header_second']].fillna(method = 'ffill')

    num = (df_heading['header_first'] == 'Prolific Data').sum()
    df_heading = df_heading.iloc[num : , :]

    ranks = {col: {ques: rank for rank, ques in enumerate(df_heading[col].unique(),1)} for col in df_heading.columns}
        
    for category, rank in ranks.items():
        df_heading[category + '_encoded'] = df_heading[category].map(rank)

    df_heading['combined'] = df_heading[['header_first_encoded','header_second_encoded','header_third_encoded']].astype(str).apply(lambda x: '-'.join(x), axis=1)

    return df_heading


def extract_ques(df, code):

    # returns all three levels of queries of the survey. where code is the header name in coded headers

    first = df.loc[df['combined'] == code, 'header_first'].values[0]
    sec = df.loc[df['combined'] == code, 'header_second'].values[0]
    third = df.loc[df['combined'] == code, 'header_third'].values[0]

    return first, sec, third


def write_excel(df, dic, combinations_name):

    # get data excel sheet
    writer = pd.ExcelWriter(combinations_name, engine='xlsxwriter')
    for iv, dv in dic["iv-dv"].items():

        # add ratings for IV
        if iv in dic["iv"]["ordinal"].keys():
            ik = "ordinal"
        else:
            ik = "nominal"

        for dvs in dv:

            dvs = [col for col in df.columns if col.startswith(dvs)]

            for one_dv in dvs:

                df_temp = pd.DataFrame()
                df_temp = pd.concat([df[iv], df[one_dv]], axis = 1)

                # Drop all rows with values to drop in any column
                df_temp = df_temp[~df_temp.isin(dic["values_to_drop"]).any(axis=1)]

                # add ratings for IV
                df_temp['Rating IV'] = df_temp[iv].map(dic["iv"][ik][iv])

                # add ratings for DV
                if one_dv[:4] in dic["dv"]["ordinal"].keys():
                    dk = "ordinal"
                else:
                    dk = "nominal"

                df_temp['Rating DV'] = df_temp[one_dv].map(dic["dv"][dk][one_dv[:4]])

                sheet_name = str(iv) + ' - ' + str(one_dv)
                df_temp.to_excel(writer, sheet_name = sheet_name, index = False)

    writer.close()


def conti_table(var1, var2, xtick_list, ytick_list, margins = True,  plot = False):

    contingency_table = pd.crosstab(var1, var2, margins = margins)

    # code to display pretty
    if margins == True:
        xlab = xtick_list + ['total']
        ylab = ytick_list + ['total']
    else:
        xlab = xtick_list
        ylab = ytick_list

    contingency_table.columns = xlab
    contingency_table.index = ylab

    # Display the contingency table
    if plot == True:
        print(tabulate(contingency_table , headers= xlab, tablefmt='grid'))

    return contingency_table






### Test Code

In [2]:
def getMetadata(df_temp, dic, df_heading, len_df): 
        
    '''
    df_temp -  is data frame 
    
    Globals
    dic - dictionary with all rankings
    df_heading - list of custom headings for the project
    df - orginal data frame
    
    Returns - a list of meta data '''

    # EXTRACT METADATA
    
    # get all titles
    IV_name = dic[df_temp.columns[0]]                                                   # IV

    _, sec, third = extract_ques(df_heading, df_temp.columns[1])
    if type(third) == str :
            DV_name = third
    else:
            DV_name = sec
    DV_name = DV_name[0] + re.sub(r'[A-Za-z]', "", DV_name[1:6])                        # DV

    # get unique values to cal dof. dof = (r-1)(c-1)
    IV_unique = len(df_temp[df_temp.columns[0]].unique())                               # Unique IV
    DV_unique = len(df_temp[df_temp.columns[1]].unique())                               # Unique DV

    # get data type of the IV and DV
    IV_dataType = "nominal" if df_temp.columns[0] in list(dic["iv"]["nominal"].keys()) else "ordinal"           # IV dataType
    DV_dataType = "nominal" if df_temp.columns[1][:4] in list(dic["dv"]["nominal"].keys()) else "ordinal"       # DV dataType

    Num_rows = int(df_temp.shape[0])                                                    # Number of rows
    Num_rows_dropped = int(len_df - Num_rows)                                           # Number of rows lost

    return IV_name, DV_name, IV_unique ,DV_unique, IV_dataType, DV_dataType, Num_rows, Num_rows_dropped


def suggest_statistical_tests(IV_dataType, DV_dataType, IV_unique, DV_unique):

    # Statistical tests based on different combinations of variables and groups
    # nominal_nominal_tests = ["Chi-square test", "Fisher's exact test"]
    # nominal_ordinal_tests = ['Kruskal-Wallis test']
    # ordinal_nominal_tests = ['Jonckheere-Terpstra test', 'Cochran-Armitage trend test']
    # ordinal_ordinal_tests = ['Linear By Liner Test', 'Kendall correlation', 'Jonckheere-Terpstra test', "Cuzick test"]

    # Check the characteristics of the variables and suggest appropriate tests
    if IV_dataType == 'nominal' and DV_dataType == 'nominal' and IV_unique >= 2 and DV_unique >= 2:
        return ["Chi-square test"]
    

    elif IV_dataType == 'nominal' and DV_dataType == 'ordinal' and IV_unique > 2 and DV_unique > 2:
        return ["Kruskal-Wallis test"]
    
    
    elif IV_dataType == 'ordinal' and DV_dataType == 'nominal' and IV_unique > 2 and DV_unique <= 2:
        return ["Cochran-Armitage trend test"]


    elif IV_dataType == 'ordinal' and DV_dataType == 'nominal' and IV_unique > 2 and DV_unique > 2:
        return ["No test due to increased complexity"]
    

    elif IV_dataType == 'ordinal' and DV_dataType == 'ordinal' and IV_unique >= 2 and DV_unique >= 2:
        return ["Linear By Linear Test", "Kendall correlation", "Jonckheere-Terpstra test", "Cuzick test"]
    else:
        return ['no test avaiable']


In [3]:
def pearsonsChi2Test(df_temp, IV_dataType, DV_dataType, significant_val):
        
    ''' PEARSON'S CHI2 TEST - for nominal DV and nominal IV
    df_temp - your dataframe with one IV and one DV
    IV_datatype - is it ordinal or nominal
    DV_ datatype - is it ordinal or nominal
    significant_val - alpha under the Null Hypothesis as a number'''

    chi_contingency_table = pd.crosstab(df_temp[df_temp.columns[0]], df_temp[df_temp.columns[1]]) # do not use margins = True as it corrupts the result
    observed = chi_contingency_table.values

    # Calculate row and column totals
    row_totals = observed.sum(axis=1)
    column_totals = observed.sum(axis=0)

    # Expected values
    expected = np.outer(row_totals, column_totals)/chi_contingency_table.sum().sum()
    chi_meet_condition = "yes as expected values more than 5" if (np.asarray(expected) < 5).sum() <= 0 else "no as expected values less than 5"       # Meets condition


    #it accepts both Rated values and original string values - treats both their values as continuous 
    chi2, chi_pvalue, chi_dof = stats.contingency.chi2_contingency(chi_contingency_table, correction = False)[:3]                                     # Chi correlation, Chi P value, Chi dof
    chi_significance = 'significant' if chi_pvalue < significant_val else 'not significant'


    # Calculate Cramer V coefficient - calculate V when you’re working with any table larger than a 2 x 2 contingency table
    chi_effect_name = "CramerV"
    cramer_dof = min(chi_contingency_table.shape) - 1
    effect_size_cramer_V = np.sqrt(chi2 / (cramer_dof * chi_contingency_table.sum().sum()))                                                           # Cramer V coeff
    cramerV_interpret = cramerV_interpretation(cramer_dof, effect_size_cramer_V)                                                                      # stat interpretation

    return chi_meet_condition, chi2, chi_pvalue, chi_significance, chi_dof, chi_effect_name, effect_size_cramer_V, cramerV_interpret, chi_contingency_table


def cramerV_interpretation(cramer_dof, number):

    '''Interpret cramer V 
    cramer_dof - dof for cramer V min( row-1 , column -1 ) of contigency table
    number - cramer V stat'''

    cramer_table = { 1 : {'weak' : (0, 0.10) , 'moderate' : (0.10, 0.30), 'strong' : (0.30, 0.50), 'very strong': (0.50, 1)}, 
                    2 : {'weak' : (0, 0.07) , 'moderate' : (0.07, 0.21), 'strong' : (0.21, 0.35), 'very strong': (0.35, 1)},
                    3 : {'weak' : (0, 0.06) , 'moderate' : (0.06, 0.17), 'strong' : (0.17, 0.29), 'very strong': (0.29, 1)}, 
                    4 : {'weak' : (0, 0.05) , 'moderate' : (0.05, 0.15), 'strong' : (0.15, 0.25), 'very strong': (0.25, 1)},
                    5 : {'weak' : (0, 0.04) , 'moderate' : (0.04, 0.13), 'strong' : (0.13, 0.22), 'very strong': (0.22, 1)}}

    if number <=0 :
        return 'no assosiation'

    for effect, range_tuple in cramer_table[cramer_dof].items():
        lower_bound, upper_bound = range_tuple
        if lower_bound <= number < upper_bound:
            return effect

    return 'value could not be found - check the cramerV table'

In [4]:
def kruskalTest(df_temp, contingency_table, significant_val): 
        
    '''KRUSKAL WALLIS - independent variable is treated as nominal
    df_temp - your dataframe with one IV and one DV
    contingency_table - contigency table computed for IV and DV without margins
    IV_datatype - is it ordinal or nominal
    DV_ datatype - is it ordinal or nominal
    significant_val - alpha under the Null Hypothesis as a number'''

    '''Null Hypothesis  H0=the independent Variable - age , income etc cause the similar response for the questions (all groups in IV originate from the same distribution and have the same median)
    Alternative Hypothesis  HA=At least one of the groups in IV generates different distribution of response (at least one group originates from a different distribution and has a different median'''

    test_name = "Kruskal-Wallis test"
    assumption_met = "populations from which the samples are drawn have similar underlying distributions"

    df_temp = df_temp.sort_values('Rating IV')
    groups = [df_temp[df_temp['Rating IV'] == val]['Rating DV'] for val in df_temp['Rating IV'].unique()]
    # the method with parsing groups is the correct method and gives correct outcomes
    kruskal_stat, kruskal_pvalue = stats.kruskal(*groups)                                                                               # Kruskal stat, Kruskal P value
    kruskal_significance = 'significant' if kruskal_pvalue < significant_val else 'not significant'                                     # Kruskal p value interpretation
    kruskal_dof = len(df_temp['Rating IV'].unique()) - 1
    
    effect_name = "epsilon squared"
    count = contingency_table.sum().sum()
    kruskal_effectsize, effect_interpretation = epsilon2_interpretation(kruskal_stat, count)                                                                   # Kruskal effect size interpretation

    return test_name, assumption_met, kruskal_stat, kruskal_pvalue, kruskal_significance, kruskal_dof, effect_name, kruskal_effectsize, effect_interpretation


def epsilon2_interpretation(num, count):

    '''Interpret Kriuskal Wallis 
    number - Kriuskal Wallis stat'''

    num = abs(num)

    number = (num * (count +1))/ (count**2 - 1)

    epsilon_table = {'very weak' : (0, 0.01) , 'weak' : (0.01, 0.04), 'moderate' : (0.04, 0.16), 'relatively strong': (0.16, 0.36), 'strong':(0.36, 0.64), 'very strong': (0.64, 1.00) }

    if number == 0 :
        return number, 'no assosiation'

    for effect, range_tuple in epsilon_table.items():
        lower_bound, upper_bound = range_tuple
        if lower_bound <= number < upper_bound:
            return number, effect

    return number, "value could not be found - check the epsilon table"

In [5]:
def cochranArmitageTest(df_temp, contingency_table, significant_val):

    test_name = "Cochran-Armitage trend test - two tailed"
    assumption_met = "IV have natural ordering"
    result = sm.stats.Table(contingency_table).test_ordinal_association()
    significance = 'significant' if result.pvalue < significant_val else 'not significant' 
    doff =  len(df_temp['Rating IV'].unique()) - 1

    effect_name = "Epsilon squared"
    count = contingency_table.sum().sum()
    effect_size, effect_interpretation = epsilon2_interpretation(result.statistic, count)     
    
    return test_name, assumption_met, result.statistic, result.pvalue, significance, doff, effect_name, effect_size, effect_interpretation


def ordinalChi2Test(df_temp, contingency_table, significant_val):

    '''LINERA TO LINEAR TEST (ORDINAL CHI SQUARE TEST) + KENDALL TAU
    df_temp - your dataframe with one IV and one DV
    contingency_table - contigency table computed for IV and DV without margins
    IV_unique, DV_unique - uniques categories in IV and DV
    chi2 - as computed in pearsonsChi2Test function
    chi_dof - as computed in pearsonsChi2Test function
    significant_val - alpha under the Null Hypothesis as a number'''


    # M2 = chi-square statistic on 1 degree of freedom based on pearson's r . The reasource used pearson's
    test_name = "linear 2 linear"
    assumption_met = "Both IV and DV are ordinal"
    df = 1
    o_chi_pearsonsr_stat, _ = stats.pearsonr(df_temp['Rating IV'], df_temp['Rating DV']) # use ranks and not the categorical values                                                   
    test_statistic = (contingency_table.sum().sum() - 1) * o_chi_pearsonsr_stat**2                                                                                                # Ochi stat (pearsons) dof = 1
    o_chi_p_pvalue = 1 - stats.chi2.cdf(test_statistic, df = df)                                                                                                                  # Ochi P value (pearson's)
    o_chi_p_significance = 'significant' if o_chi_p_pvalue < significant_val else 'not significant'                                                                     # Ochi P val intrepretation (pearsons)
    effect_name = pd.NA
    effect_size, effect_interpretation = pd.NA, pd.NA

    # m2_k = (contingency_table.sum().sum() - 1) * o_chi_kendalltau_stat**2                                                                                               # Ochi stat (kendall) dof = 1
    # o_chi_k_pvalue = 1 - stats.chi2.cdf(m2_k, df = df)                                                                                                                  # Ochi P value (kendall)
    # o_chi_k_significance = 'significant' if o_chi_k_pvalue < significant_val else 'not significant'                                                                     # Ochi P val interpretation (kendall)

    return test_name, assumption_met, test_statistic, o_chi_p_pvalue, o_chi_p_significance, df, effect_name, effect_size, effect_interpretation


In [6]:
def kendallTauTest(df_temp, IV_unique, DV_unique, significant_val):

    test_name = "Kendall Tau"
    assumption_met = "Ordinal to ordinal data"

    # M2 = chi-square statistic on 1 degree of freedom based on kendalltau r 
    variant = 'b' if IV_unique == DV_unique else 'c'
    o_chi_kendalltau_stat, kt_pvalue = stats.kendalltau(df_temp['Rating IV'], df_temp['Rating DV'], variant = variant) # use ranks and not the categorical values       # Kendall tau stat, Kendall tau P value
    kendalltau_significance = 'significant' if kt_pvalue < significant_val else 'not significant'                                                                       # Kendall tau significance
    doff = (IV_unique - 1)*(DV_unique-1)
    effect_name = "tau"
    kendalltau_effectsize = o_chi_kendalltau_stat                                                                                                                       # Kendall tau Effect size
    effect_interpretation = kendalltau_interpretation(o_chi_kendalltau_stat)    
   
    return test_name, assumption_met, o_chi_kendalltau_stat, kt_pvalue, kendalltau_significance, doff, effect_name, kendalltau_effectsize, effect_interpretation


def kendalltau_interpretation(number):

    '''Interpret Kendal tau 
    number - Kendal tau stat'''

    number = abs(number)

    kendall_table = {'very weak' : (0, 0.10) , 'weak' : (0.10, 0.20), 'moderate' : (0.20, 0.30), 'very strong': (0.30, 1)}

    if number == 0 :
        return 'no assosiation'

    for effect, range_tuple in kendall_table.items():
        lower_bound, upper_bound = range_tuple
        if lower_bound <= number < upper_bound:
            return effect

    return 'value could not be found - check the kendall table'

In [7]:
def ONStatisticTests(data_workbook, dic, df_heading):

    # Data is excel file with sheets containing combination of IV and DV and their scores
    
    df_result = pd.DataFrame(columns=["IV", "DV", "Unique IV", "Unique DV", "IV dataType", "DV dataType", "Number of rows", "Number of rows lost", "suggested tests", \
        "test_name", "assumption_met", "test_statistic", "p_value", "significance", "doff", "effect_name", "effect_size", "effect_interpretation", \
            "Chi2Test", "chi_assumption_met", "chi_stat", "chi_p_value", "chi_significance", "chi_doff", "chi_effect_name", "chi_effect_size", "chi_effect_interpretation"]) #, "chi_deviation", "chi_deviation_dof", "deviation_pvalue", "deviation_significance"]) 
                                    
    significant_val = 0.05
    len_df = 499

    # load the sheet
    excel = pd.ExcelFile(data_workbook)

    # run test on each sheet 
    for sheet_name in excel.sheet_names: 

        df_temp = pd.read_excel(data_workbook, sheet_name=sheet_name)

        # META BLOCK
        IV_name, DV_name, IV_unique ,DV_unique, IV_dataType, DV_dataType, Num_rows, Num_rows_dropped = getMetadata(df_temp, dic, df_heading, len_df)

        tests = suggest_statistical_tests(IV_dataType, DV_dataType, IV_unique, DV_unique)

        # chi test everytime
        chi_assumption_met, chi_stat, chi_p_value, chi_significance, chi_doff, chi_effect_name, chi_effect_size, \
                    chi_effect_interpretation, contingency_table = pearsonsChi2Test(df_temp, IV_dataType, DV_dataType, significant_val)
        
        # chi_deviation, chi_deviation_dof, deviation_pvalue, \
        #                 deviation_significance = 

        for test in tests:

            print(test)
            
            if test == "Kruskal-Wallis test":
                test_name, assumption_met, test_statistic, p_value, significance, doff, effect_name, effect_size, effect_interpretation = kruskalTest(df_temp, contingency_table, significant_val)

                
            elif test == "Cochran-Armitage trend test":
                test_name, assumption_met, test_statistic, p_value, significance, doff, effect_name, effect_size, effect_interpretation = cochranArmitageTest(df_temp, contingency_table, significant_val)
                

            elif test == "Linear By Linear Test":
                test_name, assumption_met, test_statistic, p_value, significance, doff, effect_name, effect_size, effect_interpretation = ordinalChi2Test(df_temp, contingency_table, significant_val)
                

            elif test == "Kendall correlation": 
                test_name, assumption_met, test_statistic, p_value, significance, doff, effect_name, effect_size, effect_interpretation = kendallTauTest(df_temp, IV_unique, DV_unique, significant_val)
                
            else:
                test_name = "No dedicated test carried out due to increased complexity. check chi results or R outputs"
                assumption_met, test_statistic, p_value, significance, doff, effect_name, effect_size, effect_interpretation = pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA

            
            lst = [IV_name, DV_name, IV_unique ,DV_unique, IV_dataType, DV_dataType, Num_rows, Num_rows_dropped, tests,\
                   test_name, assumption_met, test_statistic, p_value, significance, doff, effect_name, effect_size, effect_interpretation, \
                   "Chi2Test", chi_assumption_met, chi_stat, chi_p_value, chi_significance, chi_doff, chi_effect_name, chi_effect_size, chi_effect_interpretation] #, chi_deviation, chi_deviation_dof, deviation_pvalue, deviation_significance]
            
            df_result.loc[len(df_result)] = lst

    return df_result